In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import random
from collections import Counter

# Data

In [7]:
data_path = 'joey.npy'
if '.npy' in data_path:
    lines = str(np.load(data_path)).split('\n')

In [8]:
input_lines = list()
target_lines = list()
for line in lines[0:500]:
    if len(line):
        inp,target = line.split('\t')
        input_lines.append(inp)
        target_lines.append('<BOS> '+ target + ' <EOS>')

In [2]:
import codecs
with codecs.open("encoder_inputs.txt", "rb", encoding="utf-8", errors="ignore") as f:
    lines = f.read().split("\n")
    encoder_text = []
    for line in lines:
        data = line.split("\n")[0]
        encoder_text.append(data)
with codecs.open("decoder_inputs.txt", "rb", encoding="utf-8", errors="ignore") as f:
    lines = f.read().split("\n")
    decoder_text = []
    for line in lines:
        data = line.split("\n")[0]
        decoder_text.append(data)

# Vectors

## One-Hot Vectors

In [3]:
num_samples = 500
encoder_text = encoder_text[:num_samples]
decoder_text = decoder_text[:num_samples]
full_text = encoder_text + decoder_text

In [4]:
# from keras.preprocessing.text import Tokenizer
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(full_text)
# word_index = tokenizer.word_index

# encoder_sequences = tokenizer.texts_to_sequences(encoder_text)
# decoder_sequences = tokenizer.texts_to_sequences(decoder_text)
# decoder_output_data = np.zeros((num_samples, decoder_input_data.shape[1], len(word2index)), dtype="float32")
# for i, seqs in enumerate(decoder_input_data[0:498]):
#     for j, seq in enumerate(seqs):
#         if j > 0:
#             decoder_output_data[i][j][seq] = 1.

In [5]:
word2index = {word:index+1 for index,(word,freq) in enumerate(Counter(re.sub('\s+',' ',' '.join(full_text)).split()).items())}
word2index['<UnKnown_Token>'] = 0

In [6]:
index2word = {}
for k, v in word2index.items():
    index2word[v] = k

In [7]:
encoder_sequences = []
for sent in encoder_text:
    encoder_sequences.append([word2index[word] for word in sent.split()])

In [8]:
decoder_sequences = []
for sent in decoder_text:
    decoder_sequences.append([word2index[word] for word in sent.split()])

In [9]:
MAX = max([len(i) for i in decoder_sequences])
decoder_output_data = []
for sents in decoder_sequences[0:10]:
    sents = sents[1:]
    oneHot = [[0 for i in range(len(word2index))] for j in range(MAX)]
#     oneHot = np.zeros((MAX,len(word2index)))
    for idx,word in enumerate(sents):
        oneHot[idx][word] = 1.
    decoder_output_data.append(oneHot)
decoder_output_data = np.array(decoder_output_data)

In [10]:
from keras.preprocessing.sequence import pad_sequences
encoder_input_data = pad_sequences(encoder_sequences, dtype='int32', padding='post', truncating='post')
decoder_input_data = pad_sequences(decoder_sequences, dtype='int32', padding='post', truncating='post')

Using TensorFlow backend.
C:\Python\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Python\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Python\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Python\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning:

In [13]:
sent = 9
print(encoder_text[sent])
print([i for i in encoder_input_data[sent]])
# print([index2word[i] for i in encoder_input_data[sent]])
print()
print(decoder_text[sent])
print([i for i in decoder_input_data[sent]])
# print([index2word[i] for i in decoder_input_data[sent]])
print([[idx for idx,j in enumerate(i) if j==1] for i in decoder_output_data[sent]])
print([[index2word[idx] for idx,j in enumerate(i) if j==1] for i in decoder_output_data[sent]])  

that i am used to
[40, 6, 18, 41, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

<BOS> yeah but these guys have never seen a horse  they just jack off to clint eastwood <EOS>
[1280, 27, 112, 497, 141, 51, 187, 660, 10, 1289, 501, 147, 1290, 643, 20, 1291, 1292, 1281, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[27], [112], [497], [141], [51], [187], [660], [10], [1289], [501], [147], [1290], [643], [20], [1291], [1292], [1281], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],

## Word2vec

In [14]:
embeddings_index = {}
with open('glove.6B.50d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

print("Glove Loded!")

Glove Loded!


In [15]:
embedding_dimention = 50
def embedding_matrix_creater(embedding_dimention, word_index):
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dimention))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
          # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [16]:
embedding_matrix = embedding_matrix_creater(50, word_index=word2index)

In [17]:
from keras.layers import Embedding
from keras.layers import Input, Dense, LSTM, TimeDistributed
from keras.models import Model

In [18]:
embed_layer = Embedding(input_dim=len(word2index)+1, output_dim=50, trainable=True,)
embed_layer.build((None,))
embed_layer.set_weights([embedding_matrix])

Instructions for updating:
Colocations handled automatically by placer.


In [19]:
embed_layer.get_weights()[0][0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

# Model

In [69]:
encoder_inputs = Input(shape=(None, ), dtype='int32',)
encoder_embedding = embed_layer(encoder_inputs)
encoder_LSTM = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)

decoder_inputs = Input(shape=(None, ), dtype='int32',)
decoder_embedding = embed_layer(decoder_inputs)
decoder_LSTM = LSTM(50, return_state=True, return_sequences=True)
decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=[state_h, state_c])

decoder_dense = TimeDistributed(Dense(len(word2index), activation='softmax'))
outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], outputs)
model.compile(optimizer='adam', loss ='categorical_crossentropy', metrics = ['accuracy'])

In [70]:
model.fit([encoder_input_data[0:10], decoder_input_data[0:10]], 
                     decoder_output_data[0:10], 
                     epochs=1)

Epoch 1/1
10/10 [==============================] - 2s 179ms/step - loss: 0.7321 - acc: 0.0000e+00


In [71]:
decoder_state_input_h = Input(shape=(50,))
decoder_state_input_c = Input(shape=(50,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs2, state_h2, state_c2 = model.layers[4](model.layers[2](model.inputs[-1]), initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = model.layers[-1](decoder_outputs2)
decoder_model = Model([model.inputs[-1]] + decoder_states_inputs, [decoder_outputs2] + decoder_states2)

In [72]:
_, state_h, state_c = model.layers[3](model.layers[2](model.inputs[0]))
encoder_model = Model(model.inputs[0], [state_h, state_c])

In [73]:
input_seq = encoder_sequences[100]
[index2word[i] for i in input_seq]

['how', 'do', 'you', 'get', 'your', 'hair', 'to', 'look', 'like', 'that']

In [74]:
# Encode the input as state vectors.
states_value = encoder_model.predict(input_seq)
# Generate empty target sequence of length 1.
target_seq = np.zeros((1,1))
# Populate the first character of target sequence with the start character.
target_seq[0, 0] = word2index['<BOS>']

# Sampling loop for a batch of sequences
# (to simplify, here we assume a batch of size 1).
stop_condition = False
decoded_sentence = ''
while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # Sample a token
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index2word[sampled_token_index]
    decoded_sentence += ' '+sampled_char

    # Exit condition: either hit max length
    # or find stop character.
    if (sampled_char == '<EOS>' or
       len(decoded_sentence) > 52):
        stop_condition = True

    # Update the target sequence (of length 1).
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = sampled_token_index

    # Update states
    states_value = [h, c]

In [75]:
decoded_sentence

" proposition ' consuming ' whose freaky majesty whose"

In [ ]:
class Charachter:
    def __init__(self,):